### Open Source LLM

We can combine LLamaCPP or [Ollama integrated](https://ollama.ai) to use [Code LLaMA](https://about.fb.com/news/2023/08/code-llama-ai-for-coding/) /blog/run-code-llama-locally）.

Note：Please update `llama-cpp-python` , so that the `gguf` [format](https://github.com/abetlen/llama-cpp-python/pull/633) can be used here.

````
CMAKE_ARGS =“-DLLAMA_METAL = on”FORCE_CMAKE = 1 /Users/rlm/miniforge3/envs/llama2/bin/pip install -U llama-cpp-python --no-cache-dir
````

Check the lastest code-llama model [here](https://huggingface.co/TheBloke/CodeLlama-13B-Instruct-GGUF/tree/main).

In [5]:
!pip install -U llama-cpp-python
!pip install openai tiktoken chromadb langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 90.4 MB/s eta 0:00:00ta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 13.8 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.28-cp38-cp38-manylinux_2_31_x86_64.whl size=2113205 sha256=b4ce51d3f00900add3524f1b5ce69bc7602159f986c6ca4e7bc86ca7090370cb
  Stored in directory: /srv/users/rudxia/.cache/pip/wheels/12/12/f8/9e08a6fefe1455c9f86713e5c834fa8cfc117c64ca82c53c25
Successfully built llama-cpp-python


In [6]:
from langchain import PromptTemplate
from langchain.llms import LlamaCpp
from langchain.chains import LLMChain
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [37]:
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])
llm = LlamaCpp(
    model_path="/srv/users/rudxia/Developer_MICCAI/CodeLlama/model/codellama-13b-instruct.Q3_K_M.gguf",
    n_ctx=100000,
    n_gpu_layers=1,
    n_batch=512,
    f16_kv=False,  # MUST set to True, otherwise you will run into problem after a couple of calls
    callback_manager=callback_manager,
    verbose=True,
)

llama_model_loader: loaded meta data with 20 key-value pairs and 363 tensors from /srv/users/rudxia/Developer_MICCAI/CodeLlama/model/codellama-13b-instruct.Q3_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = codellama_codellama-13b-instruct-hf
llama_model_loader: - kv   2:                       llama.context_length u32              = 16384
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 5120
llama_model_loader: - kv   4:                          llama.block_count u32              = 40
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 13824
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_

## **Prompt after formatting**

In [38]:
question="Generate a C++ program that accepts numeric input from the user and maintains a record of previous user inputs with timestamps. Ensure the program sorts the user inputs in ascending order based on the provided numeric input. Enhance the program to display timestamps along with the sorted user inputs."
prompt=PromptTemplate.from_template(question)

default_chain=LLMChain(llm=llm, prompt=prompt, verbose=True)
default_chain.run(prompt=question)



> Entering new LLMChain chain...
Prompt after formatting:
Generate a C++ program that accepts numeric input from the user and maintains a record of previous user inputs with timestamps. Ensure the program sorts the user inputs in ascending order based on the provided numeric input. Enhance the program to display timestamps along with the sorted user inputs.


#include <iostream>
#include <iomanip>
using namespace std;
int main() {
	cout<<"Please enter the number:";
	double num, temp = 0;
	cin>>num;
	if (num % 2 == 0) {
		temp = num / 2;
	} else if ((num + 1) % 2 == 0)) {
		temp = ((num + 1))) / 2;
	}
	cout<<"The number is: " << temp << endl;





return 0;
















> Finished chain.



llama_print_timings:        load time =    2303.34 ms
llama_print_timings:      sample time =      41.32 ms /   154 runs   (    0.27 ms per token,  3727.01 tokens per second)
llama_print_timings: prompt eval time =    2301.16 ms /    64 tokens (   35.96 ms per token,    27.81 tokens per second)
llama_print_timings:        eval time =   21380.00 ms /   153 runs   (  139.74 ms per token,     7.16 tokens per second)
llama_print_timings:       total time =   24370.36 ms


'\n\n#include <iostream>\n#include <iomanip>\nusing namespace std;\nint main() {\n\tcout<<"Please enter the number:";\n\tdouble num, temp = 0;\n\tcin>>num;\n\tif (num % 2 == 0) {\n\t\ttemp = num / 2;\n\t} else if ((num + 1) % 2 == 0)) {\n\t\ttemp = ((num + 1))) / 2;\n\t}\n\tcout<<"The number is: " << temp << endl;\n\n\n\n\n\nreturn 0;\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n'

## **Prompt directly**

In [54]:
prompt = """
Question: Generate a C++ program that accepts numeric input from the user and maintains a record of previous user inputs with timestamps. Ensure the program sorts the user inputs in ascending order based on the provided numeric input. Enhance the program to display timestamps along with the sorted user inputs.
"""
llm(prompt)

Comment: Please clarify your specific problem or provide additional details to highlight exactly what you need. As it's currently written, it's hard to tell exactly what you're asking.
Answer: \begin{code}
#include <iostream>
using namespace std;
int main() {
    int num1 = 0;
    cout << "Enter a number: ";
    cin >> num1;
    if (num1 > 0) {
        cout << "\nThe number is positive.";
    } else if ((num1 < 0)) {
        cout << "\nThe number is negative.";
    } else {
        cout << "\nThe number is zero.";
    }
}
\end{code}


llama_print_timings:        load time =     309.13 ms
llama_print_timings:      sample time =      47.67 ms /   165 runs   (    0.29 ms per token,  3461.22 tokens per second)
llama_print_timings: prompt eval time =    2692.54 ms /    69 tokens (   39.02 ms per token,    25.63 tokens per second)
llama_print_timings:        eval time =   23341.78 ms /   164 runs   (  142.33 ms per token,     7.03 tokens per second)
llama_print_timings:       total time =   26691.10 ms


'Comment: Please clarify your specific problem or provide additional details to highlight exactly what you need. As it\'s currently written, it\'s hard to tell exactly what you\'re asking.\nAnswer: \\begin{code}\n#include <iostream>\nusing namespace std;\nint main() {\n    int num1 = 0;\n    cout << "Enter a number: ";\n    cin >> num1;\n    if (num1 > 0) {\n        cout << "\\nThe number is positive.";\n    } else if ((num1 < 0)) {\n        cout << "\\nThe number is negative.";\n    } else {\n        cout << "\\nThe number is zero.";\n    }\n}\n\\end{code}'